# Supervised Learning : Regression
## Peer Review Project 2

This project uses the publically available **IBM HR Analytics Employee Attrition & Performance** hosted on Kaggle 

[https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset](https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset)

---

## Report Summary

This is a report on the application of various linear regression models in an attempt to predict the attrition of employees basing on their performance data. 

### Data set details



### Objectives 



### Linear regression models used in the project




### Results 
1. Most informative features of the dataset 
2. Best performing model 
3. General performance of models


### Conclusion 
1. Flaws in the dataset
2. Flaws in the model/Objectives


---

# Code below this point

Beyond this point is the code used in the project for details of the analysis summarised in the section above. 